In [41]:
from rapidfuzz import process, fuzz
import pandas as pd
from tqdm import tqdm
import re

In [94]:
# Function to find the first two numbers in the address
def find_first_number(address):
    match = re.search(r'\d+', address)
    return int(match.group()) if match else None

def find_first_two_numbers(address):
    matches = re.findall(r'\d+', address)
    return matches[:2] if len(matches) >= 2 else None


In [43]:
# Function to normalize the text to check the similarity as best as it could
def normalize_text(text):
    # Ensure the text is in string format
    text_string = str(text)
    # Remove all special characters
    normalized_text = re.sub(r'[^\w\s]', '', text_string)
    return normalized_text.upper()  # Convert to uppercase for consistency

In [96]:
def check_no_word_matches(field1, field2):
    # Split both fields into sets of words, converting everything to lowercase to ensure case-insensitive comparison
    words_field1 = set(field1.lower().split())
    words_field2 = set(field2.lower().split())
    
    # Check for intersection
    if words_field1.isdisjoint(words_field2):
        return True  # No words match
    else:
        return False  # There is at least one matching word

In [97]:
# testing

df_fnb = pd.read_csv('csv_data\fnb_list_part_2_cleaned.csv', encoding='utf-8')
df_halal = pd.read_csv('csv_data\premis_makanan_list_part_4_cleaned.csv', encoding='utf-8')

# Combine 'Name' and 'Address' into a single string for matching
df_fnb['Combined'] = df_fnb.apply(lambda row: normalize_text(row['Name'] + ' ' + normalize_text(row['Address'])), axis=1)
df_halal['Combined'] = df_halal.apply(lambda row: normalize_text(row['Premise Name'] + ' ' + normalize_text(row['Address'])), axis=1)

THRESHOLD = 81.5
#SKIP_CHECKING_THRESHOLD = 89
MAX_MATCHES = 30
matches = []

for index, fnb_row in tqdm(df_fnb.iterrows(), total=df_fnb.shape[0], desc="Searching for matches"):
    # Extract the combined field for the current row
    combined_fnb = fnb_row['Combined']
    
    # Perform fuzzy matching against the combined fields of df_halal
    best_match = process.extractOne(combined_fnb, df_halal['Combined'].tolist(), scorer=fuzz.token_set_ratio)
    
    # Check if the best match score meets the threshold
    if best_match and best_match[1] >= THRESHOLD:
        
        # Retrieve the matching row from df_halal
        halal_row = df_halal[df_halal['Combined'] == best_match[0]].iloc[0]

        normalized_fnb_name = normalize_text(fnb_row['Name'])
        normalized_fnb_address =  normalize_text(fnb_row['Address'])
        normalized_halal_name = normalize_text(halal_row['Premise Name'])
        normalized_halal_address =  normalize_text(halal_row['Address'])

        # Check if halal row address length is extremely shorter compared to fnb row address
        if len(fnb_row['Address'])/3 > len(halal_row['Address']):
            if len(matches) >= MAX_MATCHES:
                break
            else:
                continue

        # Check if both fnb name and halal nama really no common words at all 
        if check_no_word_matches(normalized_fnb_name,normalized_halal_name) == True:
            if len(matches) >= MAX_MATCHES:
                break
            else:
                continue
    
        # Check if fnb name and premise name are exactly the same
        if (normalized_fnb_name.replace(' ', '') == normalized_halal_name.replace(' ', '')):#fuzz.token_set_ratio(normalized_fnb_name, normalized_halal_name) >= 99) and ):
            matches.append((fnb_row['Name'], fnb_row['Address'], halal_row['Premise Name'], halal_row['Address'], best_match[1]))
            if len(matches) >= MAX_MATCHES:
                break
            else:
                continue

        # if best_match and best_match[1] < SKIP_CHECKING_THRESHOLD:
        # Check whether if the first two detected numbers in fnb address is equivalent to premise address
        first_two_numbers_in_fnb_address = find_first_two_numbers(normalized_fnb_address)
        first_two_numbers_in_halal_address = find_first_two_numbers(normalized_halal_address)
        # Check whether if the first number in fnb address is equivalent to premise address
        first_number_in_fnb_address = find_first_number(normalized_fnb_address)
        first_number_in_halal_address = find_first_number(normalized_halal_address)

        if (first_two_numbers_in_fnb_address != None and first_two_numbers_in_halal_address != None):
            if first_two_numbers_in_fnb_address == first_two_numbers_in_halal_address:
                matches.append((fnb_row['Name'], fnb_row['Address'], halal_row['Premise Name'], halal_row['Address'], best_match[1]))
            # Break if maximum matches are found
            if len(matches) >= MAX_MATCHES:
                break
            else:
                continue
        
        if (first_number_in_fnb_address != None and first_number_in_halal_address != None):
            if first_number_in_fnb_address == first_number_in_halal_address:
                matches.append((fnb_row['Name'], fnb_row['Address'], halal_row['Premise Name'], halal_row['Address'], best_match[1]))
                # Break if maximum matches are found
            if len(matches) >= MAX_MATCHES:
                break
            else:
                continue
        # else:
        #     matches.append((fnb_row['Name'], fnb_row['Address'], halal_row['Premise Name'], halal_row['Address'], best_match[1]))
        # Break if maximum matches are found
        if len(matches) >= MAX_MATCHES:
            break

# Print matches
for match in matches[:MAX_MATCHES]:
    print(f"FNB Name: {match[0]}") 
    print(f"FNB Address: {match[1]}")
    print(f"Premise Name: {match[2]}")
    print(f"Premise Address: {match[3]}")
    print(f"Match Score: {match[4]}\n")


Searching for matches:  11%|█▏        | 459/4070 [00:15<02:05, 28.87it/s]

FNB Name: 4Fingers Crispy Chicken @ Puchong Utama
FNB Address: Pusat No, 20 (GF, Jalan PU 7/2, Bandar Puchong Utama, 47140 Puchong, Selangor
Premise Name: 4FINGERS CRISPY CHICKEN PUCHONG UTAMA
Premise Address: 20 (GF), JALAN PU 7/2,  TAMAN PUCHONG UTAMA, PUCHONG
Match Score: 94.82758620689656

FNB Name: A&W BMC Mall
FNB Address: Lot G-16, Ground Floor BMC Mall, Jalan Temenggung 21/9, Bandar Mahkota Cheras, 43200 Cheras, Selangor
Premise Name: A&W BMC MALL
Premise Address: LOT G-16, GROUND FLOOR BMC MALL, JALAN TEMENGGUNG 21/9, BANDAR MAHKOTA CHERAS
Match Score: 100.0

FNB Name: A&W C180 Cheras Selatan
FNB Address: 3A & 5,Jalan C180/1, Bt 11 Cheras, 43200 Balakong, Selangor
Premise Name: A&W C180 CHERAS SELATAN
Premise Address: 3A & 5,JALAN C180/1, BT 11 CHERAS
Match Score: 100.0

FNB Name: A&W Citta Mall
FNB Address: Citta Mall, G-10 Ground Floor, Jalan PJU 1a/48, Pusat Perdagangan Dana 1, 47301 Petaling Jaya, Selangor
Premise Name: A&W CITTA MALL
Premise Address: G-10, GROUND FLOOR, C

In [104]:
def merge_row(fnb_index, halal_row):
    columns = ['Premise Name', 'Company Brand Name', 'Halal Certification Expiry Date', 'Halal Certification Body']
    for col in columns:
        df_fnb.at[fnb_index, col] = halal_row[col]
        

In [105]:
# Official Flow

df_fnb = pd.read_csv('csv_data\fnb_list_part_2_cleaned.csv', encoding='utf-8')
df_halal = pd.read_csv('csv_data\premis_makanan_list_part_4_cleaned.csv', encoding='utf-8')

# Combine 'Name' and 'Address' into a single string for matching
df_fnb['Combined'] = df_fnb.apply(lambda row: normalize_text(row['Name'] + ' ' + normalize_text(row['Address'])), axis=1)
df_halal['Combined'] = df_halal.apply(lambda row: normalize_text(row['Premise Name'] + ' ' + normalize_text(row['Address'])), axis=1)

for col in ['Premise Name','Company Brand Name','Halal Certification Expiry Date','Halal Certification Body']:
    df_fnb[col] = None

THRESHOLD = 81.5

for fnb_index, fnb_row in tqdm(df_fnb.iterrows(), total=df_fnb.shape[0], desc="Searching for matches"):
    # Extract the combined field for the current row
    combined_fnb = fnb_row['Combined']
    
    # Perform fuzzy matching against the combined fields of df_halal
    best_match = process.extractOne(combined_fnb, df_halal['Combined'].tolist(), scorer=fuzz.token_set_ratio)
    
    # Check if the best match score meets the threshold
    if best_match and best_match[1] >= THRESHOLD:
        
        # Retrieve the matching row from df_halal
        halal_row = df_halal[df_halal['Combined'] == best_match[0]].iloc[0]

        normalized_fnb_name = normalize_text(fnb_row['Name'])
        normalized_fnb_address =  normalize_text(fnb_row['Address'])
        normalized_halal_name = normalize_text(halal_row['Premise Name'])
        normalized_halal_address =  normalize_text(halal_row['Address'])

        # Check if halal row address length is extremely shorter compared to fnb row address
        if len(fnb_row['Address'])/3 > len(halal_row['Address']):
            continue

        # Check if both fnb name and halal nama really no common words at all 
        if check_no_word_matches(normalized_fnb_name,normalized_halal_name) == True:
            continue
    
        # Check if fnb name and premise name are exactly the same
        if (normalized_fnb_name.replace(' ', '') == normalized_halal_name.replace(' ', '')):#fuzz.token_set_ratio(normalized_fnb_name, normalized_halal_name) >= 99) and ):
            merge_row(fnb_index, halal_row)
            continue

        # Check whether if the first two detected numbers in fnb address is equivalent to premise address
        first_two_numbers_in_fnb_address = find_first_two_numbers(normalized_fnb_address)
        first_two_numbers_in_halal_address = find_first_two_numbers(normalized_halal_address)
        # Check whether if the first number in fnb address is equivalent to premise address
        first_number_in_fnb_address = find_first_number(normalized_fnb_address)
        first_number_in_halal_address = find_first_number(normalized_halal_address)

        if (first_two_numbers_in_fnb_address != None and first_two_numbers_in_halal_address != None):
            if first_two_numbers_in_fnb_address == first_two_numbers_in_halal_address:
                merge_row(fnb_index, halal_row)
            continue
        
        if (first_number_in_fnb_address != None and first_number_in_halal_address != None):
            if first_number_in_fnb_address == first_number_in_halal_address:
                merge_row(fnb_index, halal_row)
            continue



Searching for matches: 100%|██████████| 4070/4070 [02:24<00:00, 28.21it/s]


In [107]:
display( df_fnb)
display( df_fnb.shape)
display( df_fnb.describe())
display( df_fnb.info)

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Combined,Premise Name,Company Brand Name,Halal Certification Expiry Date,Halal Certification Body
0,#leleh,NaN,['Cafe'],Cheras,MyTOWN Shopping Centre,"Seksyen, 90, Jalan Cochrane, Cheras, 55100 Kua...",03-2720 1772,https://www.google.com/maps/place/%23leleh/dat...,NaN,NaN,NaN,LELEH SEKSYEN 90 JALAN COCHRANE CHERAS 55100 K...,None,None,None,None
1,1 Plus 1 Coffee • Fruit ( Café ),1 Plus 1 Coffee,['Cafe'],Puchong,NaN,"1, Jalan Puteri 7/11, Bandar Puteri, 47100 Puc...",016-934 7388,https://www.google.com/maps/place/1+Plus+1+Cof...,NaN,http://web.facebook.com/profile.pho?id=1000844...,https://grab.onelink.me/2695613898?pid=inappsh...,1 PLUS 1 COFFEE FRUIT CAFÉ 1 JALAN PUTERI 7...,None,None,None,None
2,10 Pasento Puchong,10 Pasento Puchong,['Cafe'],Puchong,Floor 1 · Hotel Sri Sutra,"1st Floor, 10, Jalan Kenari 6, Bandar Puchong ...",NaN,https://www.google.com/maps/place/10+Pasento+P...,NaN,https://www.instagram.com/10_pasento/,NaN,10 PASENTO PUCHONG 1ST FLOOR 10 JALAN KENARI 6...,None,None,None,None
3,103 Coffee,103 Coffee,['Cafe'],Seri Kembangan,NaN,"103-G, Jalan Radin Bagus, Bandar Baru Sri Peta...",012-519 7103,https://www.google.com/maps/place/103+Coffee/d...,http://103coffeeworkshop.beepit.com/,https://103coffee.com/,NaN,103 COFFEE 103G JALAN RADIN BAGUS BANDAR BARU ...,None,None,None,None
4,128 Thai,128 Thai,['Western Food'],Subang Jaya,NaN,"42, Jalan SS 15/4c, Ss 15, 47500 Subang Jaya, ...",016-233 0609,https://www.google.com/maps/place/128+Thai/dat...,NaN,https://128thai.business.site/?m=true,https://wa.me/message/EY3JJ7HNEGQ4G1,128 THAI 42 JALAN SS 154C SS 15 47500 SUBANG J...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4065,麻婆婆麻辣香锅·冒菜（SS15店）,麻婆婆麻辣香锅,['Chinese Food'],Subang Jaya,NaN,"Selangor, Subang Jaya, 1nd Floor 70, Jalan SS ...",NaN,https://www.google.com/maps/place/%E9%BA%BB%E5...,NaN,NaN,https://www.foodpanda.my/restaurant/shtw/mapop...,麻婆婆麻辣香锅冒菜SS15店 SELANGOR SUBANG JAYA 1ND FLOOR ...,None,None,None,None
4066,黄凳仔,黄凳仔,['Chinese Food'],Seri Kembangan,NaN,"13-25, Jalan BS 3/6, Taman Bukit Serdang, 4330...",NaN,https://www.google.com/maps/place/%E9%BB%84%E5...,NaN,NaN,NaN,黄凳仔 1325 JALAN BS 36 TAMAN BUKIT SERDANG 43300...,None,None,None,None
4067,龍焱麻辣香锅 Loong Yan Mala Hotpot,龍焱麻辣香锅 Loong Yan Mala Hotpot,['Chinese Food'],Seri Kembangan,NaN,"8, Jalan PSK 4, Pusat Perdagangan Seri Kembang...",03-8958 5880,https://www.google.com/maps/place/%E9%BE%8D%E7...,https://wa.me//60389585880,NaN,https://www.foodpanda.my/restaurant/l4d2/loong...,龍焱麻辣香锅 LOONG YAN MALA HOTPOT 8 JALAN PSK 4 PUS...,None,None,None,None
4068,𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀 𝗣𝗮𝘁𝗶𝘀𝘀𝗲𝗿𝗶𝗲,𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀 𝗣𝗮𝘁𝗶𝘀𝘀𝗲𝗿𝗶𝗲,['Dessert'],Petaling Jaya,NaN,"27-1, Jalan PJU 5/11, Dataran Sunway, Kota Dam...",017-600 6710,https://www.google.com/maps/place/%F0%9D%97%96...,https://www.champignonspatisserie.com/en_MY/,https://www.champignonspatisserie.com/en_MY/,NaN,𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀 𝗣𝗮𝘁𝗶𝘀𝘀𝗲𝗿𝗶𝗲 271 JALAN PJU 511 DATAR...,None,None,None,None


(4070, 16)

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Combined,Premise Name,Company Brand Name,Halal Certification Expiry Date,Halal Certification Body
count,4070,4067,3926,4070,957,4055,3451,4070,624,2083,1383,4070,174,174,173,174
unique,4070,3969,90,5,485,3769,3214,4070,516,1665,1269,4070,171,54,65,1
top,#leleh,Baskin-Robbins,['Chinese Food'],Subang Jaya,MyTOWN Shopping Centre,"Ss 15, 47500 Subang Jaya, Selangor",1-300-88-2525,https://www.google.com/maps/place/%23leleh/dat...,https://kfc.com.my/,https://kfc.com.my/,http://kfc.com.my/,LELEH SEKSYEN 90 JALAN COCHRANE CHERAS 55100 K...,A&W DAMEN MALL,GERBANG ALAF RESTAURANTS SDN BHD (FORMERLY KNO...,2024-06-15,JABATAN KEMAJUAN ISLAM MALAYSIA (JAKIM)
freq,1,13,543,924,22,6,22,1,22,42,24,1,2,19,16,174


<bound method DataFrame.info of                                   Name               Simplified Name  \
0                               #leleh                           NaN   
1     1 Plus 1 Coffee • Fruit ( Café )              1 Plus 1 Coffee    
2                   10 Pasento Puchong            10 Pasento Puchong   
3                           103 Coffee                    103 Coffee   
4                             128 Thai                      128 Thai   
...                                ...                           ...   
4065                 麻婆婆麻辣香锅·冒菜（SS15店）                       麻婆婆麻辣香锅   
4066                               黄凳仔                           黄凳仔   
4067      龍焱麻辣香锅 Loong Yan Mala Hotpot  龍焱麻辣香锅 Loong Yan Mala Hotpot   
4068            𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀 𝗣𝗮𝘁𝗶𝘀𝘀𝗲𝗿𝗶𝗲        𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀 𝗣𝗮𝘁𝗶𝘀𝘀𝗲𝗿𝗶𝗲   
4069              𝗧𝗮𝘀𝘁𝗲 𝗕𝘆 𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀          𝗧𝗮𝘀𝘁𝗲 𝗕𝘆 𝗖𝗵𝗮𝗺𝗽𝗶𝗴𝗻𝗼𝗻𝘀   

              Category            Area                 Located In  \
0             ['Cafe']          Ch

In [108]:
df_fnb.to_csv('csv_data\fnb_list_part_6_merged.csv', index=False, encoding='utf-8')